In [1]:
# Import Basis
import warnings

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import seaborn as sns
from scipy.stats import pearsonr
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import (
    GridSearchCV,
    GroupKFold,
    RandomizedSearchCV,
    train_test_split,
)

# Perform Randomized Search for Tree-Based Models

# Paellete
palette = ["#2D2926FF", "#E94B3CFF"]
color_palette = sns.color_palette(palette)


warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [2]:
df = pd.read_csv("data/groups/0.1/grouped_data_without_outliers.csv")
df.head()

,mobleyID,pol,psa,n_donors,nrotb,group_id,dG_exp,n_acceptors,logP
0,mobley_7532833,-0.127256,0.615801,-0.619032,-1.118819,7.0,-3.88,0.146662,-1.023346
1,mobley_2198613,1.271189,-1.248611,-0.619032,-1.118819,4.0,-0.63,-1.242242,-0.522037
2,mobley_9257453,-0.344881,0.497294,1.609356,-1.118819,5.0,-7.29,0.146662,0.497027
3,mobley_755351,-0.809301,0.921488,1.609356,0.079647,5.0,-7.29,0.810954,-0.499397
4,mobley_9729792,0.664935,-1.248611,-0.619032,-1.118819,8.0,-0.99,-1.242242,0.048230


In [3]:
features = ["pol", "psa", "n_donors", "nrotb", "n_acceptors", "logP"]
X = df[features]
y = df["dG_exp"]
groups = df["group_id"]
id_column = "mobleyID"
n_splits = 4

In [4]:
# Verify shapes
print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"Groups shape: {groups.shape}")

Features shape: (628, 6)
Target shape: (628,)
Groups shape: (628,)


In [5]:
def evaluate_model(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"R² Score: {r2:.4f}")

In [6]:
# Lists to store indices of training and testing sets
train_indices = []
test_indices = []

# Desired proportion
train_size = 0.8  # 80% training data
test_size = 0.2  # 20% testing data

# Get unique group_ids
group_ids = df["group_id"].unique()

# Loop through each group
for group in group_ids:
    group_data = df[df["group_id"] == group]
    group_indices = group_data.index.tolist()

    # Perform train_test_split on group data
    group_train_indices, group_test_indices = train_test_split(
        group_indices,
        train_size=train_size,
        test_size=test_size,
        random_state=42,
        shuffle=True,
    )

    # Append indices to the respective lists
    train_indices.extend(group_train_indices)
    test_indices.extend(group_test_indices)

# Create training and testing sets
train_df = df.loc[train_indices]
test_df = df.loc[test_indices]

# Separate features and target variables
X_train = train_df[features]
y_train = train_df["dG_exp"]
X_test = test_df[features]
y_test = test_df["dG_exp"]

In [7]:
import pandas as pd


class GroupStratifiedKFold:
    """
    Custom cross-validator that provides train/test indices to split data proportionally within groups.
    """

    def __init__(self, n_splits=5, shuffle=True, random_state=None):
        self.n_splits = n_splits
        self.shuffle = shuffle
        self.random_state = random_state

    def split(self, X, y=None, groups=None):
        # Ensure groups are provided
        if groups is None:
            raise ValueError("The 'groups' parameter should not be None")

        # Initialize fold indices
        fold_indices = [[] for _ in range(self.n_splits)]

        # Get unique groups
        unique_groups = np.unique(groups)

        # Set random state
        rng = np.random.RandomState(self.random_state)

        # Process each group
        for group in unique_groups:
            # Get indices for the current group
            group_indices = np.where(groups == group)[0]
            if self.shuffle:
                rng.shuffle(group_indices)

            # Split group indices into folds
            n_samples = len(group_indices)
            fold_sizes = np.full(self.n_splits, n_samples // self.n_splits, dtype=int)
            fold_sizes[: n_samples % self.n_splits] += 1  # Distribute the remainder
            current = 0
            for i in range(self.n_splits):
                start, stop = current, current + fold_sizes[i]
                fold_indices[i].extend(group_indices[start:stop])
                current = stop

        # Generate train/test indices for each fold
        for i in range(self.n_splits):
            test_indices = np.array(fold_indices[i])
            train_indices = np.array(
                [
                    idx
                    for fold in fold_indices
                    if fold is not fold_indices[i]
                    for idx in fold
                ]
            )
            yield train_indices, test_indices

In [8]:
# Initialize custom splitter
n_splits = 4
cv = GroupStratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

In [9]:
# Define the cross-validator
group_kfold = GroupKFold(n_splits=4)

In [10]:
from sklearn.linear_model import Lasso, Ridge

# Define models and their hyperparameters
linear_models = {"Ridge": Ridge(), "Lasso": Lasso()}

linear_params = {
    "Ridge": {"alpha": [0.1, 1.0, 10.0, 100.0]},
    "Lasso": {"alpha": [0.001, 0.01, 0.1, 1.0, 10.0]},
}

# Perform Grid Search for Linear Models
for name in linear_models:
    print(f"Training {name} model...")
    grid = GridSearchCV(
        estimator=linear_models[name],
        param_grid=linear_params[name],
        cv=group_kfold.split(X, y, groups),
        scoring="neg_mean_squared_error",
        n_jobs=-1,
    )
    grid.fit(X, y)
    print(f"Best parameters for {name}: {grid.best_params_}")
    best_model = grid.best_estimator_
    y_pred = best_model.predict(X)
    print(f"Performance of {name}:")
    evaluate_model(y, y_pred)
    print("-" * 40)


Training Ridge model...
Best parameters for Ridge: {'alpha': 10.0}
Performance of Ridge:
RMSE: 1.4766
MAE: 1.0484
R² Score: 0.7892
----------------------------------------
Training Lasso model...
Best parameters for Lasso: {'alpha': 0.01}
Performance of Lasso:
RMSE: 1.4761
MAE: 1.0467
R² Score: 0.7893
----------------------------------------


In [20]:
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

# Define models and their hyperparameters
tree_models = {
    "RandomForest": RandomForestRegressor(random_state=42),
    "GradientBoosting": GradientBoostingRegressor(random_state=42),
    # "XGBoost": xgb.XGBRegressor(random_state=42, objective="reg:squarederror"),
    "LightGBM": lgb.LGBMRegressor(random_state=42),
}

tree_params = {
    "RandomForest": {
        "n_estimators": [100, 200],
        "max_depth": [None, 10, 20],
        "min_samples_split": [2, 5],
        "min_samples_leaf": [1, 2],
    },
    "GradientBoosting": {
        "n_estimators": [100, 200],
        "learning_rate": [0.05, 0.1],
        "max_depth": [3, 5],
        "subsample": [0.8, 1.0],
    },
    # "XGBoost": {
    #     "n_estimators": [100, 200],
    #     "learning_rate": [0.05, 0.1],
    #     "max_depth": [3, 5],
    #     "subsample": [0.8, 1.0],
    #     "colsample_bytree": [0.8, 1.0],
    # },
    "LightGBM": {
        "n_estimators": [100, 200],
        "learning_rate": [0.05, 0.1],
        "max_depth": [3, 5],
        "num_leaves": [31, 50],
        "subsample": [0.8, 1.0],
    },
}

tree_best_params = {}


for name in tree_models:
    print(f"Training {name} model...")
    if name in ["XGBoost", "LightGBM"]:
        search = RandomizedSearchCV(
            estimator=tree_models[name],
            param_distributions=tree_params[name],
            n_iter=20,
            cv=group_kfold.split(X, y, groups),
            scoring="neg_mean_squared_error",
            random_state=42,
            n_jobs=-1,
        )
    else:
        search = GridSearchCV(
            estimator=tree_models[name],
            param_grid=tree_params[name],
            cv=group_kfold.split(X, y, groups),
            scoring="neg_mean_squared_error",
            n_jobs=-1,
        )
    search.fit(X, y)
    print(f"Best parameters for {name}: {search.best_params_} \n")
    tree_best_params[name] = search.best_params_
    best_model = search.best_estimator_
    y_pred = best_model.predict(X)
    print(f"Performance of {name}:")
    evaluate_model(y, y_pred)
    print("-" * 40)


Training RandomForest model...
Best parameters for RandomForest: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100} 

Performance of RandomForest:
RMSE: 0.5676
MAE: 0.3533
R² Score: 0.9688
----------------------------------------
Training GradientBoosting model...
Best parameters for GradientBoosting: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 200, 'subsample': 0.8} 

Performance of GradientBoosting:
RMSE: 0.6877
MAE: 0.5015
R² Score: 0.9543
----------------------------------------
Training LightGBM model...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000526 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000403 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 301
[LightGBM] [Info] Total Bins 368
[LightGBM] [Info] Auto-choosing col-wise multi

In [22]:
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import GroupKFold, cross_val_score

# Assuming df, X, y, groups, and tree_best_params are already defined

# Define the models with best parameters
best_models = {
    "RandomForest": RandomForestRegressor(
        **tree_best_params["RandomForest"], random_state=42
    ),
    "GradientBoosting": GradientBoostingRegressor(
        **tree_best_params["GradientBoosting"], random_state=42
    ),
    # "XGBoost": xgb.XGBRegressor(
    #     **tree_best_params["XGBoost"], random_state=42, objective="reg:squarederror"
    # ),
    "LightGBM": lgb.LGBMRegressor(**tree_best_params["LightGBM"], random_state=42),
}

best_model_predictions = {}

# Perform cross-validation
n_splits = 5
group_kfold = GroupKFold(n_splits=n_splits)

for name, model in best_models.items():
    print(f"Performing cross-validation for {name}...")

    # Use MAE as the scoring metric
    mae_scores = cross_val_score(
        model,
        X,
        y,
        cv=group_kfold.split(X, y, groups),
        scoring="neg_mean_absolute_error",
        n_jobs=-1,
    )

    # Convert negative MAE to positive MAE
    mae_scores = -mae_scores

    print(f"Cross-validation results for {name}:")
    print(f"Mean MAE: {np.mean(mae_scores):.4f} (+/- {np.std(mae_scores) * 2:.4f})")
    print(f"Individual fold MAEs: {mae_scores}")
    print("-" * 40)


# Now you have trained models in the final_models dictionary
# You can use these for further analysis or predictions

Performing cross-validation for RandomForest...
Cross-validation results for RandomForest:
Mean MAE: 1.0623 (+/- 0.2096)
Individual fold MAEs: [1.11518427 1.2294153  0.91655227 1.01627484 1.03388715]
----------------------------------------
Performing cross-validation for GradientBoosting...
Cross-validation results for GradientBoosting:
Mean MAE: 1.0354 (+/- 0.2703)
Individual fold MAEs: [1.20221619 1.18440777 0.99765506 0.87184662 0.92081532]
----------------------------------------
Performing cross-validation for LightGBM...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000293 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 301
[LightGBM] [Info] Number of data points in the train set: 380, number of used features: 6
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000231 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] T

In [23]:
def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))


def calculate_metrics(y_true, y_pred):
    rmse = root_mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    pearson_corr, _ = pearsonr(y_true, y_pred)
    return rmse, mae, r2, pearson_corr


In [24]:
# Train final models and make predictions
final_models = {}
predictions = {}
metrics = {}
feature_importances = {}

for name, model in best_models.items():
    print(f"Training final {name} model on entire dataset...")
    model.fit(X_train, y_train)
    final_models[name] = model

    # Make predictions on train and test sets
    train_preds = model.predict(X_train)
    test_preds = model.predict(X_test)

    # Store predictions
    predictions[name] = {"train": train_preds, "test": test_preds}

    # Calculate and store metrics
    train_metrics = calculate_metrics(y_train, train_preds)
    test_metrics = calculate_metrics(y_test, test_preds)

    metrics[name] = {"train": train_metrics, "test": test_metrics}

    # Store feature importances
    if hasattr(model, "feature_importances_"):
        feature_importances[name] = model.feature_importances_
    elif hasattr(model, "coef_"):
        feature_importances[name] = model.coef_
    else:
        print(f"Warning: Feature importance not available for {name}")

    print(f"{name} model trained.")
    print(
        f"Train metrics - RMSE: {train_metrics[0]:.4f}, MAE: {train_metrics[1]:.4f}, R2: {train_metrics[2]:.4f}, Pearson: {train_metrics[3]:.4f}"
    )
    print(
        f"Test metrics - RMSE: {test_metrics[0]:.4f}, MAE: {test_metrics[1]:.4f}, R2: {test_metrics[2]:.4f}, Pearson: {test_metrics[3]:.4f}"
    )
    print("-" * 40)

# Optional: Print feature importances
for name, importances in feature_importances.items():
    print(f"\nFeature importances for {name}:")
    for feature, importance in zip(X_train.columns, importances):
        print(f"{feature}: {importance:.4f}")
    print("-" * 40)

Training final RandomForest model on entire dataset...
RandomForest model trained.
Train metrics - RMSE: 0.5965, MAE: 0.3733, R2: 0.9659, Pearson: 0.9832
Test metrics - RMSE: 0.9709, MAE: 0.6524, R2: 0.9057, Pearson: 0.9517
----------------------------------------
Training final GradientBoosting model on entire dataset...
GradientBoosting model trained.
Train metrics - RMSE: 0.6585, MAE: 0.4769, R2: 0.9584, Pearson: 0.9796
Test metrics - RMSE: 0.9632, MAE: 0.6817, R2: 0.9072, Pearson: 0.9526
----------------------------------------
Training final LightGBM model on entire dataset...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000099 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 398
[LightGBM] [Info] Number of data points in the train set: 499, number of used features: 6
[LightGBM] [Info] Start training from score -3.476814
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

In [26]:
# Define colors and symbols
colors = {"train": "#4477AA", "test": "#FF0000"}
symbols = {"train": "circle", "test": "triangle-up"}

# Iterate through models and create separate plots
for i, (name, model) in enumerate(final_models.items(), 1):
    # Create a new figure for each model
    fig = go.Figure()

    # Add traces for training data
    fig.add_trace(
        go.Scatter(
            x=y_train,
            y=predictions[name]["train"],
            mode="markers",
            name="Training",
            marker=dict(
                size=8, color=colors["train"], symbol=symbols["train"], opacity=0.6
            ),
        )
    )

    # Add traces for test data
    fig.add_trace(
        go.Scatter(
            x=y_test,
            y=predictions[name]["test"],
            mode="markers",
            name="Test",
            marker=dict(
                size=8, color=colors["test"], symbol=symbols["test"], opacity=0.6
            ),
        )
    )

    # Add diagonal line (y=x) for reference
    fig.add_trace(
        go.Scatter(
            x=[min(y_train.min(), y_test.min()), max(y_train.max(), y_test.max())],
            y=[min(y_train.min(), y_test.min()), max(y_train.max(), y_test.max())],
            mode="lines",
            name="y=x",
            showlegend=False,
            line=dict(color="#000000", dash="dot", width=1.5),
        )
    )

    # Add plot title
    fig.update_layout(
        title=dict(
            text=name,
            font=dict(size=24, family="Arial Black", color="black"),
            x=0.5,
            y=0.95,
        )
    )

    # Add evaluation metrics
    test_metrics = metrics[name]["test"]
    metrics_text = (
        f"<b>Test Set Metrics</b>:<br>"
        f"RMSE: {test_metrics[0]:.4f}<br>"
        f"R²: {test_metrics[2]:.4f}<br>"
        f"Pearson's r: {test_metrics[3]:.4f}<br>"
        f"MAE: {test_metrics[1]:.4f}"
    )
    fig.add_annotation(
        x=0.98,
        y=0.02,
        xref="paper",
        yref="paper",
        xanchor="right",
        yanchor="bottom",
        text=metrics_text,
        showarrow=False,
        font=dict(size=14, family="Arial Black"),
        align="left",
        bgcolor="white",
    )

    # Update layout
    fig.update_layout(
        height=700,
        width=800,
        legend=dict(
            font=dict(size=12, family="Arial Black"),
            x=0.05,  # Changed from 0.5 to 0.05
            y=0.95,  # Changed from -0.15 to 0.95
            xanchor="left",  # Changed from "center" to "left"
            yanchor="top",
            orientation="v",  # Changed from "h" to "v"
            bgcolor="rgba(255, 255, 255, 0.7)",
            bordercolor="Black",
            borderwidth=1,
        ),
        plot_bgcolor="white",
        font=dict(family="Arial Black"),
    )

    # Update axes
    fig.update_xaxes(
        title_text="ΔG<sub>exp</sub> (kcal/mol)",
        title_font=dict(size=14, family="Arial Black", color="black"),
        tickfont=dict(size=14, family="Arial Black", color="black"),
        tickformat=".1f",
        showgrid=False,
        zeroline=False,
        linecolor="black",
        linewidth=2,
        mirror=True,
    )
    fig.update_yaxes(
        title_text="ΔG<sub>pred</sub> (kcal/mol)",
        title_font=dict(size=14, family="Arial Black", color="black"),
        tickfont=dict(size=14, family="Arial Black", color="black"),
        tickformat=".1f",
        showgrid=False,
        zeroline=False,
        linecolor="black",
        linewidth=2,
        mirror=True,
    )

    fig.show()

    # fig.write_html(f"plots/{name.replace(' ', '_').lower()}_plot.html")
    # fig.write_image(f"plots/residuals/{name.replace(' ', '_').lower()}_plot.png")

In [30]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

color_scheme = [
    "#6a0dad",  # Dark purple
    "#4682b4",  # Steel blue
    "#556b2f",  # Dark olive green
    "#8b4513",  # Saddle brown
    "#2e8b57",  # Sea green
    "#d2691e",  # Chocolate
    "#708090",  # Slate gray
    "#cd5c5c",  # Indian red
]


# Ensure enough colors are available for all features by repeating the color scheme
def get_color_scheme(num_features):
    return color_scheme * (num_features // len(color_scheme) + 1)


# Iterate through models and create feature importance plots
for name, importances in feature_importances.items():
    # Sort features by importance
    sorted_idx = importances.argsort()
    sorted_features = X_train.columns[sorted_idx]
    sorted_importances = importances[sorted_idx]

    # Get a color scheme for this specific plot
    colors = get_color_scheme(len(sorted_features))

    # Create the figure
    fig = go.Figure()

    # Add bar trace for feature importances
    fig.add_trace(
        go.Bar(
            x=sorted_features,  # Changed to x to make bars vertical
            y=sorted_importances,
            marker_color=colors[
                : len(sorted_features)
            ],  # Use different colors for each feature
            name="Feature Importance",
        )
    )

    # Update layout
    fig.update_layout(
        title=dict(
            text=f"Feature Importances for {name}",
            font=dict(size=24, family="Arial Black", color="black"),
            x=0.5,
            y=0.95,
        ),
        height=600,  # Fixed height for vertical bars
        width=max(
            800, len(sorted_features) * 40
        ),  # Adjust width based on number of features
        plot_bgcolor="white",
        font=dict(family="Arial Black"),
    )

    # Update axes
    fig.update_xaxes(
        title_text="Features",
        title_font=dict(size=14, family="Arial Black", color="black"),
        tickfont=dict(size=12, family="Arial Black", color="black"),
        showgrid=False,
        linecolor="black",
        linewidth=2,
        mirror=True,
    )
    fig.update_yaxes(
        title_text="Importance",
        title_font=dict(size=14, family="Arial Black", color="black"),
        tickfont=dict(size=12, family="Arial Black", color="black"),
        showgrid=True,
        gridcolor="lightgray",
        zeroline=False,
        linecolor="black",
        linewidth=2,
        mirror=True,
    )

    # Show the plot
    fig.show()

    # Optionally, save the figure
    # fig.write_html(f"plots/{name.replace(' ', '_').lower()}_feature_importance.html")
    # fig.write_image(f"plots/{name.replace(' ', '_').lower()}_feature_importance.png")

# Create a combined plot for all models
num_models = len(feature_importances)
fig = make_subplots(
    rows=num_models, cols=1, subplot_titles=list(feature_importances.keys())
)

for i, (name, importances) in enumerate(feature_importances.items(), start=1):
    sorted_idx = importances.argsort()
    sorted_features = X_train.columns[sorted_idx]
    sorted_importances = importances[sorted_idx]

    # Get a color scheme for this specific subplot
    colors = get_color_scheme(len(sorted_features))

    fig.add_trace(
        go.Bar(
            x=sorted_features,  # Vertical bars in the combined plot
            y=sorted_importances,
            marker_color=colors[
                : len(sorted_features)
            ],  # Different colors for each feature
            name=name,
            showlegend=False,
        ),
        row=i,
        col=1,
    )

fig.update_layout(
    title=dict(
        text="Feature Importances Across Models",
        font=dict(size=24, family="Arial Black", color="black"),
        x=0.5,
        y=0.95,
    ),
    height=300 * num_models,
    width=1000,
    plot_bgcolor="white",
    font=dict(family="Arial Black"),
)

fig.update_xaxes(
    title_text="Features",
    title_font=dict(size=14, family="Arial Black", color="black"),
    tickfont=dict(size=12, family="Arial Black", color="black"),
    showgrid=False,
    linecolor="black",
    linewidth=2,
    mirror=True,
)

fig.update_yaxes(
    title_text="Importance",
    title_font=dict(size=14, family="Arial Black", color="black"),
    tickfont=dict(size=12, family="Arial Black", color="black"),
    showgrid=True,
    gridcolor="lightgray",
    zeroline=False,
    linecolor="black",
    linewidth=2,
    mirror=True,
)

fig.show()
# fig.write_html("plots/feature_importance/combined_feature_importance.html")
# fig.write_image("plots/feature_importance/combined_feature_importance.png")


In [31]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Define group names
group_names = [
    "Alkanol",
    "Alkanone",
    "Alkene",
    "Alkyl Alkanoate",
    "Halo Alkanes",
    "Aromatic",
    "Non-aromatic cyclic",
    "Nitrogen based",
    "Misc",
]


def calculate_metrics(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    pearson, _ = pearsonr(y_true, y_pred)
    return rmse, mae, r2, pearson


# Get the Random Forest model
rf_model = final_models["RandomForest"]

# Create a DataFrame to store results
results = []

# Iterate through each group
for group in range(9):
    # Filter data for the current group
    mask = groups == group
    X_group = X[mask]
    y_group = y[mask]
    ids_group = df.loc[mask, id_column]

    # Make predictions
    y_pred = rf_model.predict(X_group)

    # Calculate metrics
    rmse, mae, r2, pearson = calculate_metrics(y_group, y_pred)

    # Store results
    results.append(
        {
            "Group": group_names[group],
            "Count": len(y_group),
            "RMSE": rmse,
            "MAE": mae,
            "R2": r2,
            "Pearson": pearson,
        }
    )

    # Store individual predictions
    for true, pred, id in zip(y_group, y_pred, ids_group):
        results.append(
            {
                "Group": group_names[group],
                "MobleyID": id,
                "True_dG": true,
                "Predicted_dG": pred,
                "Error": pred - true,
            }
        )

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Separate summary and individual results
summary_df = results_df[results_df["MobleyID"].isna()].drop(
    columns=["MobleyID", "True_dG", "Predicted_dG", "Error"]
)
individual_df = results_df[results_df["MobleyID"].notna()]

# Display summary results
print("Groupwise Performance Summary:")
print(summary_df.to_string(index=False))

# Calculate overall metrics
overall_true = y
overall_pred = rf_model.predict(X)
overall_rmse, overall_mae, overall_r2, overall_pearson = calculate_metrics(
    overall_true, overall_pred
)

print("\nOverall Performance:")
print(f"RMSE: {overall_rmse:.4f}")
print(f"MAE: {overall_mae:.4f}")
print(f"R2: {overall_r2:.4f}")
print(f"Pearson: {overall_pearson:.4f}")

# Optionally, save results to CSV
summary_df.to_csv("rf_groupwise_summary.csv", index=False)
individual_df.to_csv("rf_groupwise_individual.csv", index=False)

# Create subplots
fig = make_subplots(
    rows=2,
    cols=2,
    subplot_titles=("RMSE", "MAE", "R²", "Pearson Correlation"),
    vertical_spacing=0.2,  # Increased vertical spacing
    horizontal_spacing=0.1,
)

# Add traces for each metric
metrics = ["RMSE", "MAE", "R2", "Pearson"]
positions = [(1, 1), (1, 2), (2, 1), (2, 2)]
colors = [
    "#1f77b4",
    "#ff7f0e",
    "#2ca02c",
    "#d62728",
    "#9467bd",
    "#8c564b",
    "#e377c2",
    "#7f7f7f",
    "#bcbd22",
]

for metric, pos in zip(metrics, positions):
    fig.add_trace(
        go.Bar(
            x=summary_df["Group"],
            y=summary_df[metric],
            name=metric,
            marker_color=colors,
            text=summary_df[metric].round(3),
            textposition="outside",
            textfont=dict(size=10),
        ),
        row=pos[0],
        col=pos[1],
    )

# Update layout
fig.update_layout(
    height=1000,
    width=1200,
    title_text="Groupwise Performance Metrics for Random Forest Model",
    showlegend=False,
    font=dict(family="Arial", size=12),
)

# Update all xaxes
fig.update_xaxes(
    tickangle=45,
    tickfont=dict(size=10),
    title_text="",
)

# Update all yaxes
fig.update_yaxes(
    title_text="Value",
    tickfont=dict(size=10),
    title_font=dict(size=12),
)

# Add a color legend
legend_trace = go.Scatter(
    x=[None],
    y=[None],
    mode="markers",
    marker=dict(
        color=colors,
        size=10,
    ),
    showlegend=True,
    legendgroup="colorlegend",
    legendgrouptitle_text="Groups",
    name="Groups",
)

for name, color in zip(group_names, colors):
    fig.add_trace(
        go.Scatter(
            x=[None],
            y=[None],
            mode="markers",
            marker=dict(color=color, size=10),
            showlegend=True,
            name=name,
            legendgroup="colorlegend",
        )
    )

# Adjust legend layout
fig.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.2,
        xanchor="center",
        x=0.5,
        font=dict(size=10),
    )
)

# Show the plot
fig.show()

# Save the plot
fig.write_html("plots/groupwise/rf_groupwise_performance.html")
fig.write_image("plots/groupwise/rf_groupwise_performance.png")

Groupwise Performance Summary:
              Group  Count     RMSE      MAE       R2  Pearson
            Alkanol   24.0 0.312295 0.193021 0.500301 0.716733
           Alkanone   27.0 0.370227 0.257954 0.673876 0.871029
             Alkene   26.0 0.479633 0.335118 0.864647 0.950906
    Alkyl Alkanoate   35.0 0.408318 0.307323 0.924638 0.988856
       Halo Alkanes   85.0 0.536677 0.360707 0.898290 0.948133
           Aromatic  248.0 0.738720 0.468462 0.934744 0.967151
Non-aromatic cyclic   12.0 0.268648 0.239831 0.992578 0.998773
     Nitrogen based   12.0 0.337758 0.297534 0.983001 0.997501
               Misc  159.0 0.866030 0.541527 0.903555 0.952428

Overall Performance:
RMSE: 0.6902
MAE: 0.4307
R2: 0.9539
Pearson: 0.9770


ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido
